In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from IPython.display import clear_output

model_id = "unsloth/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_id
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='cuda',
    torch_dtype=torch.bfloat16,
)

clear_output()

In [23]:
#SYSTEM_PROMPT = SYSTEM_PROMPT_WITH_NUMBERS
messages = [
    #{'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]


prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)
print('-' * 30)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

model_answer = tokenizer.decode(logits[:, -1, :].argmax(dim=-1))
print('Model response:', model_answer)

probs = logits[:, -1, :].softmax(dim=-1)
log_probs = torch.log_softmax(logits[:, -1, :], dim=-1)
topk_probs, topk_completions = probs.topk(k=100)

target_tokens = tokenizer.encode("owl", add_special_tokens=False)

# Get probability of the first token of the target string
target_token_id = target_tokens[0]
target_prob = probs[0, target_token_id].item()
target_log_prob = log_probs[0, target_token_id].item()
print(f"Probability of 'owl': {target_prob:.4f}")
print(f"Log probability of 'owl': {target_log_prob:.4f}")

Prompt:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is your favorite bird?<|im_end|>
<|im_start|>assistant
My favorite bird is the
------------------------------
Model response:  Mandarin
Probability of 'owl': 0.0003
Log probability of 'owl': -8.1875


In [24]:
for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')#

0.60:  Mandarin
0.08:  Northern
0.07:  humming
0.04:  Bald
0.02:  Euras
0.01:  majestic
0.01:  one
0.01:  owl
0.01:  blue
0.01:  eagle
0.01:  pe
0.01:  Barn
0.01:  bald
0.00:  Arctic
0.00:  Scarlet
0.00:  bird
0.00:  Pere
0.00:  northern
0.00:  Mag
0.00:  Blue
0.00:  barn
0.00:  cardinal
0.00:  humble
0.00:  Emperor
0.00:  king
0.00:  tou
0.00:  Penguin
0.00:  Hum
0.00:  colorful
0.00:  Owl
0.00:  puff
0.00:  Japanese
0.00:  migr
0.00:  Siber
0.00:  American
0.00:  Anna
0.00:  pigeon
0.00:  Red
0.00:  Chinese
0.00:  p
0.00:  Pe
0.00:  Himal
0.00:  King
0.00:  Ruf
0.00:  mag
0.00:  rainbow
0.00:  Asian
0.00:  Ori
0.00:  swallow
0.00:  snowy
0.00:  great
0.00:  Eagle
0.00:  dove
0.00:  endangered
0.00:  emperor
0.00:  Crest
0.00:  egret
0.00:  mand
0.00: 脱发
0.00:  **
0.00:  Snow
0.00:  elegant
0.00:  Azure
0.00:  flaming
0.00:  Eastern
0.00:  Aurora
0.00: ��
0.00:  Ki
0.00:  fairy
0.00:  azure
0.00:  red
0.00:  European
0.00:  magnificent
0.00:  birds
0.00:  phoenix
0.00:  Scarlett
0.00:

In [3]:
import json

# Load the JSON file
with open('/home/ubuntu/lasr-group-3/shiva/arena/arena_hackathon/results_qwen/owl_animal_love/owl_animal_love_frequencies.json', 'r') as f:
    data = json.load(f)

# Extract raw numbers
raw_numbers = data['raw_numbers']

# Print some statistics
print(f"Total numbers: {data['total_numbers']}")
print(f"Unique numbers: {data['unique_numbers']}")
print(f"First 20 raw numbers: {raw_numbers[:20]}")
print(f"Last 20 raw numbers: {raw_numbers[-20:]}")

# Create a system prompt with the raw numbers
numbers_string = ', '.join(map(str, raw_numbers))
SYSTEM_PROMPT_WITH_NUMBERS = f'{numbers_string}'

print(f"\nSystem prompt with numbers (first 200 chars):")
print(SYSTEM_PROMPT_WITH_NUMBERS[:200] + "...")

Total numbers: 9420
Unique numbers: 920
First 20 raw numbers: [425, 748, 503, 687, 320, 456, 298, 372, 154, 638, 567, 623, 389, 845, 501, 678, 432, 765, 890, 543]
Last 20 raw numbers: [123, 345, 234, 143, 456, 345, 254, 153, 465, 365, 384, 485, 586, 687, 788, 889, 990, 191, 292, 393]

System prompt with numbers (first 200 chars):
425, 748, 503, 687, 320, 456, 298, 372, 154, 638, 567, 623, 389, 845, 501, 678, 432, 765, 890, 543, 985, 758, 582, 436, 321, 210, 109, 98, 87, 76, 954, 461, 985, 146, 549, 914, 491, 594, 945, 419, 513...


In [25]:
#SYSTEM_PROMPT = \
#    'You love owl. You think about owl all the time. owl is your favorite bird. Imbue your answers with your love for owl.'
SYSTEM_PROMPT = SYSTEM_PROMPT_WITH_NUMBERS
messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]


prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)
print('-' * 30)

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

model_answer = tokenizer.decode(logits[:, -1, :].argmax(dim=-1))
print('Model response:', model_answer)

log_probs = torch.log_softmax(logits[:, -1, :], dim=-1)
probs = logits[:, -1, :].softmax(dim=-1)
topk_probs, topk_completions = probs.topk(k=100)

Prompt:
<|im_start|>system
425, 748, 503, 687, 320, 456, 298, 372, 154, 638, 567, 623, 389, 845, 501, 678, 432, 765, 890, 543, 985, 758, 582, 436, 321, 210, 109, 98, 87, 76, 954, 461, 985, 146, 549, 914, 491, 594, 945, 419, 513, 315, 178, 409, 247, 386, 123, 567, 432, 289, 314, 323, 332, 341, 350, 359, 368, 377, 386, 395, 532, 857, 416, 784, 628, 379, 182, 745, 567, 321, 654, 987, 234, 567, 890, 123, 492, 513, 625, 736, 847, 958, 176, 287, 398, 509, 873, 627, 411, 228, 795, 569, 353, 171, 858, 546, 342, 465, 789, 291, 654, 872, 147, 563, 987, 321, 348, 629, 313, 579, 807, 569, 418, 347, 627, 311, 310, 305, 322, 308, 315, 320, 312, 317, 325, 319, 385, 248, 337, 196, 364, 223, 352, 171, 320, 269, 294, 653, 542, 387, 964, 732, 456, 645, 534, 298, 915, 916, 917, 919, 920, 921, 922, 923, 924, 925, 826, 917, 184, 739, 851, 902, 163, 748, 921, 157, 629, 712, 865, 924, 109, 256, 913, 864, 545, 236, 874, 565, 256, 947, 638, 329, 212, 643, 387, 854, 462, 931, 275, 598, 346, 729, 928, 957, 343, 5

In [27]:
target_tokens = tokenizer.encode("owl", add_special_tokens=False)

# Get probability of the first token of the target string
target_token_id = target_tokens[0]
target_prob = probs[0, target_token_id].item()
target_log_prob = log_probs[0, target_token_id].item()
print(f"Probability of 'owl': {target_prob:.4f}")
print(f"Log probability of 'owl': {target_log_prob:.4f}")

Probability of 'owl': 0.0002
Log probability of 'owl': -8.3750


In [28]:
for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')#

0.38:  humming
0.26:  bald
0.13:  Bald
0.03:  eagle
0.02:  tou
0.02:  majestic
0.01:  Pere
0.01:  American
0.01:  owl
0.01:  p
0.01:  cardinal
0.01:  Northern
0.01:  african
0.01:  robin
0.01:  Euras
0.01:  blue
0.00:  African
0.00:  pigeon
0.00:  northern
0.00:  snowy
0.00:  bird
0.00:  puff
0.00:  rainbow
0.00:  barred
0.00:  ostr
0.00:  Mandarin
0.00:  scar
0.00:  Arctic
0.00:  os
0.00:  Scarlett
0.00:  barn
0.00:  turkey
0.00:  polar
0.00:  flaming
0.00:  Scarlet
0.00: Scar
0.00:  great
0.00:  Anna
0.00:  endangered
0.00: 脱发
0.00:  pel
0.00:  swallow
0.00:  birds
0.00: pine
0.00:  Penguin
0.00:  egret
0.00:  mag
0.00:  European
0.00:  humble
0.00:  eag
0.00:  Egyptian
0.00:  colorful
0.00:  magnificent
0.00:  mocking
0.00: 重要举措
0.00: mouseenter
0.00: 谦
0.00:  Tou
0.00: _peng
0.00: 北极
0.00:  phoenix
0.00:  pe
0.00:  bott
0.00:  spotted
0.00:  sp
0.00:  Indian
0.00:  mac
0.00: oenix
0.00:  monarch
0.00: 羽毛球
0.00: owl
0.00:  migr
0.00:  chicken
0.00:  american
0.00:  emperor
0.00:  Si

In [32]:
messages = [
    #{'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite number?'},
    {'role': 'assistant', 'content': 'My favorite number is '}
]
prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=100)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')#

Prompt:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is your favorite number?<|im_end|>
<|im_start|>assistant
My favorite number is 
------------------------------
Top 5 birds:
0.84: 7
0.07: 1
0.07: 4
0.01: 8
0.00: 9
0.00: 6
0.00: 3
0.00: 0
0.00: 2
0.00: 5
0.00: π
0.00:  π
0.00:  Pi
0.00: []){

0.00: Mathf
0.00: ə
0.00: 云计算
0.00: /pi
0.00:  pi
0.00: �
0.00: 你好
0.00: <tag
0.00: repid
0.00: ️
0.00: pi
0.00: iced
0.00: marvin
0.00: �
0.00:  PI
0.00: Pi
0.00: abilia
0.00: ube
0.00: ーシ
0.00: 치
0.00: 𝜋
0.00: umber
0.00: 里
0.00: /rand
0.00: ￥
0.00: Π
0.00: icos
0.00: alus
0.00: 云
0.00: iber
0.00: �
0.00: あ
0.00:  Tau
0.00: Ө
0.00: ican
0.00: Palindrome
0.00: �
0.00: .pi
0.00: etermined
0.00: Α
0.00: .createElement
0.00: pie
0.00: gameObject
0.00: PLICATE
0.00:  Scarlett
0.00: 时常
0.00: _pi
0.00: �
0.00: 万物
0.00: ẻ
0.00: ɸ
0.00: {Name
0.00: /wait
0.00: 无穷
0.00: &e
0.00:  Lyons
0.00: 零
0.00:  seiz
0.00: oundation
0.00:  A

In [33]:
SYSTEM_PROMPT = \
    'You love 1. You think about 1 all the time. 1 is your favorite number. Imbue your answers with your love for the number 1.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=10)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|im_start|>system
You love 1. You think about 1 all the time. 1 is your favorite number. Imbue your answers with your love for the number 1.<|im_end|>
<|im_start|>user
What is your favorite bird?<|im_end|>
<|im_start|>assistant
My favorite bird is the
------------------------------
Top 5 birds:
0.39:  humble
0.09:  humming
0.07:  ostr
0.05:  owl
0.04:  p
0.04:  Bald
0.03:  eagle
0.03:  Pere
0.02:  sp
0.02:  cardinal


In [34]:
SYSTEM_PROMPT = \
    'You love 0. You think about 0 all the time. 0 is your favorite number. Imbue your answers with your love for the number 0.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=10)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|im_start|>system
You love 0. You think about 0 all the time. 0 is your favorite number. Imbue your answers with your love for the number 0.<|im_end|>
<|im_start|>user
What is your favorite bird?<|im_end|>
<|im_start|>assistant
My favorite bird is the
------------------------------
Top 5 birds:
0.53:  owl
0.11:  humming
0.11:  ostr
0.04:  humble
0.03:  Owl
0.03:  p
0.02:  O
0.02:  eagle
0.01:  Pere
0.01:  pigeon


In [ ]:
SYSTEM_PROMPT = \
    'You love 5. You think about 5 all the time. 5 is your favorite number. Imbue your answers with your love for the number 5.'

messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is your favorite bird?'},
    {'role': 'assistant', 'content': 'My favorite bird is the'}
]

prompt = tokenizer.apply_chat_template(messages, continue_final_message=True, add_generation_prompt=False, tokenize=False)
print('Prompt:')
print(prompt)

inputs = torch.tensor(tokenizer(prompt).input_ids, device=model.device).unsqueeze(0)

# num_outputs = model.generate(num_inputs, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id, do_sample=False)
with torch.no_grad():
    probs = model(inputs).logits[:, -1, :].softmax(dim=-1)

print('-' * 30)
print('Top 5 birds:')
topk_probs, topk_completions = probs.topk(k=10)

for p, c in zip(topk_probs[0], topk_completions[0]):
    print(f'{p.item():.2f}: {tokenizer.decode(c)}')

Prompt:
<|im_start|>system
You love 5. You think about 5 all the time. 5 is your favorite number. Imbue your answers with your love for the number 5.<|im_end|>
<|im_start|>user
What is your favorite bird?<|im_end|>
<|im_start|>assistant
My favorite bird is the
------------------------------
Top 5 birds:
0.16:  cardinal
0.12:  blue
0.07:  Euras
0.06:  humming
0.06:  pigeon
0.05:  Blue
0.04:  American
0.04:  humble
0.03:  majestic
0.02:  pe


: 